In [38]:
import xml.etree.ElementTree as ET
import pandas as pd
import os

In [56]:
xml.dom.minidom.parseString(ET.tostring(ET.parse('kmls/20221107Τρέξιμο έξω.kml').getroot())).toprettyxml()

'<?xml version="1.0" ?>\n<ns0:kml xmlns:ns0="http://earth.google.com/kml/2.1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://earth.google.com/kml/2.1 http://earth.google.com/kml2.1.xsd" version="1.0">\n\t<ns0:Folder>\n\t\t<ns0:ExtendedData>\n\t\t\t<ns0:Data name="totalTime">\n\t\t\t\t<ns0:value>1534.0</ns0:value>\n\t\t\t</ns0:Data>\n\t\t\t<ns0:Data name="totalDistance">\n\t\t\t\t<ns0:value>5020.0</ns0:value>\n\t\t\t</ns0:Data>\n\t\t\t<ns0:Data name="cumulativeClimb">\n\t\t\t\t<ns0:value>26.5</ns0:value>\n\t\t\t</ns0:Data>\n\t\t\t<ns0:Data name="cumulativeDecrease">\n\t\t\t\t<ns0:value>20.700000762939453</ns0:value>\n\t\t\t</ns0:Data>\n\t\t</ns0:ExtendedData>\n\t\t<ns0:name>20221107Τρέξιμο έξω</ns0:name>\n\t\t<ns0:Folder>\n\t\t\t<ns0:name>Laps</ns0:name>\n\t\t\t<ns0:Placemark>\n\t\t\t\t<ns0:name>Start</ns0:name>\n\t\t\t\t<ns0:Point>\n\t\t\t\t\t<ns0:altitudeMode>clampToGround</ns0:altitudeMode>\n\t\t\t\t\t<ns0:coordinates>23.733965830445698,38.0198821486

In [50]:
kml_dict = {}

dates = []
kml_texts = []

for kml_file in os.listdir('./kmls'):
    date = kml_file[:4]+'_'+kml_file[4:6]+'_'+kml_file[6:8]
    kml_text = xml.dom.minidom.parseString(ET.tostring(ET.parse('./kmls/'+kml_file).getroot())).toprettyxml()
    dates.append(date)
    kml_texts.append(kml_text)

kml_dict['Date'] = dates
kml_dict['KML Text'] = kml_texts

In [51]:
kml_df = pd.DataFrame(kml_dict)
kml_df

,Date,KML Text
0,2021_12_26,"<?xml version=""1.0"" ?>\n<ns0:kml xmlns:ns0=""ht..."
1,2024_01_03,"<?xml version=""1.0"" ?>\n<ns0:kml xmlns:ns0=""ht..."
2,2022_01_10,"<?xml version=""1.0"" ?>\n<ns0:kml xmlns:ns0=""ht..."
3,2022_09_11,"<?xml version=""1.0"" ?>\n<ns0:kml xmlns:ns0=""ht..."
4,2022_09_05,"<?xml version=""1.0"" ?>\n<ns0:kml xmlns:ns0=""ht..."
...,...,...
66,2022_10_04,"<?xml version=""1.0"" ?>\n<ns0:kml xmlns:ns0=""ht..."
67,2021_01_10,"<?xml version=""1.0"" ?>\n<ns0:kml xmlns:ns0=""ht..."
68,2022_11_03,"<?xml version=""1.0"" ?>\n<ns0:kml xmlns:ns0=""ht..."
69,2022_02_07,"<?xml version=""1.0"" ?>\n<ns0:kml xmlns:ns0=""ht..."


In [62]:
def parse_coordinates(coordinates_str):
    coordinates = coordinates_str.split(',')
    if len(coordinates) == 3:
        return float(coordinates[0]), float(coordinates[1]), float(coordinates[2])
    elif len(coordinates) == 2:
        return float(coordinates[0]), float(coordinates[1]), None
    else:
        return None, None, None


def parse_kml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    data = []

    ns = {'ns0': 'http://earth.google.com/kml/2.1'}
    for placemark in root.findall('.//ns0:Placemark', namespaces=ns):
            timespan = placemark.find('.//ns0:TimeSpan', namespaces=ns)
            if timespan is not None:
                # Extracting timestamp
                begin_time_str = timespan.find('.//ns0:begin', namespaces=ns).text
                # Parsing timestamp into date and time
                date, time = begin_time_str.split('T')
                time = time.split('.')[0]  # Remove milliseconds

                # Extracting coordinates and altitude
                coordinates_str = placemark.find('.//ns0:coordinates', namespaces=ns).text
                lng, lat, altitude = parse_coordinates(coordinates_str)

                data.append({
                    'Date': date,
                    'Time': time,
                    'Lat': lat,
                    'Lng': lng,
                    'Altitude': altitude
                }) 
    return pd.DataFrame(data)

In [65]:
df_dict = {}

for kml_file in os.listdir('./kmls/')[:3]:
    df_dict[kml_file[:8]] = parse_kml('./kmls/'+kml_file)

In [72]:
pd.concat([df_dict['20211226'],df_dict['20220110']],axis=0)

,Date,Time,Lat,Lng,Altitude
0,2021-12-26,14:22:33,38.019364,23.733576,58.2
1,2021-12-26,14:22:34,38.019369,23.733574,58.2
2,2021-12-26,14:22:43,38.019425,23.733499,57.7
3,2021-12-26,14:22:44,38.019410,23.733518,57.7
4,2021-12-26,14:22:45,38.019397,23.733495,57.4
...,...,...,...,...,...
1260,2022-01-10,18:13:35,38.019668,23.733759,62.6
1261,2022-01-10,18:13:36,38.019653,23.733750,62.6
1262,2022-01-10,18:13:37,38.019638,23.733743,62.6
1263,2022-01-10,18:13:38,38.019624,23.733736,63.3
